<center><img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective">Objective</h3>
<br><strong style="color: #000000;"><em>Change publicly accessible of RDS DB Snapshots to private</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Secure-Publicly-accessible-Amazon-RDS-Snapshot"><u>Secure Publicly accessible Amazon RDS Snapshot</u></h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<p><br>1)<a href="#2" target="_self" rel="noopener"> Get publicly accessible DB snapshots</a><br>2)<a href="#2" target="_self" rel="noopener"> Change the public access to private</a></p>

<h3 id="List-all-AWS-Regions">List all AWS Regions</h3>
<p>This action fetches all AWS Regions to execute Step 1👇. This action will only execute if no <code>region</code> is provided.</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following ouput: <code>region</code></p>
</blockquote>

In [7]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from typing import Dict, List
import pprint

from beartype import beartype
@beartype
def aws_list_all_regions_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_list_all_regions(handle) -> List:
    """aws_list_all_regions lists all the AWS regions

        :type handle: object
        :param handle: Object returned from Task Validate

        :rtype: Result List of result
    """

    result = handle.aws_cli_command("aws ec2 describe-regions --all-regions --query 'Regions[].{Name:RegionName}' --output text")
    if result is None or result.returncode != 0:
        print("Error while executing command : {}".format(result))
        return str()
    result_op = list(result.stdout.split("\n"))
    list_region = [x for x in result_op if x != '']
    return list_region


task = Task(Workflow())
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not region",
    "condition_result": true
    }''')
task.configure(outputName="region")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_all_regions, lego_printer=aws_list_all_regions_printer, hdl=hdl, args=args)

<h3 id="Get-publicly-accessible-DB-snapshots"><a id="2" target="_self" rel="nofollow"></a>Get publicly accessible DB snapshots</h3>
<p>Using unSkript's Get Publicly Accessible DB Snapshots in RDS action we will fetch all the publicly accessible snapshots from the list of manual DB snapshots.</p>
<blockquote>
<p>This action takes the following parameters: <code>region(Optional)</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>all_snapshots</code></p>
</blockquote>

In [8]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict, Optional, Tuple
from pydantic import BaseModel, Field
from unskript.legos.utils import CheckOutput, CheckOutputStatus
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
from unskript.legos.aws.aws_filter_all_manual_database_snapshots.aws_filter_all_manual_database_snapshots import aws_get_manual_database_snapshots
import pprint


from beartype import beartype
@beartype
def aws_get_publicly_accessible_db_snapshots_printer(output):
    if output is None:
        return
    pprint.pprint(output.json())


@beartype
def aws_get_publicly_accessible_db_snapshots(handle, region: str=None) -> CheckOutput:
    """aws_get_publicly_accessible_db_snapshots lists of publicly accessible db_snapshot_identifier.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: string
        :param region: Region of the RDS.

        :rtype: Object with status, result having publicly accessible Snapshots Identifier in RDS, error
    """
    manual_snapshots_list=[]
    result=[]
    all_regions = [region]
    if region is None or not region:
        all_regions = aws_list_all_regions(handle=handle)
    try:
        for r in all_regions:
            snapshots_dict = {}
            output = aws_get_manual_database_snapshots(handle=handle, region=r)
            snapshots_dict["region"]=r
            snapshots_dict["snapshot"]=output
            manual_snapshots_list.append(snapshots_dict)
    except Exception as error:
        return CheckOutput(status=CheckOutputStatus.RUN_EXCEPTION,
                           objects=[],
                           error=error.__str__())
    for all_snapshots in manual_snapshots_list:
        try:
            ec2Client = handle.client('rds', region_name=all_snapshots['region'])
            for each_snapshot in all_snapshots['snapshot']:
                response = ec2Client.describe_db_snapshot_attributes(DBSnapshotIdentifier=each_snapshot)
                db_attribute = response["DBSnapshotAttributesResult"]
                for value in db_attribute['DBSnapshotAttributes']:
                    p_dict={}
                    if "all" in value["AttributeValues"]:
                        p_dict["region"] = all_snapshots['region']
                        p_dict["open_snapshot"] = db_attribute['DBSnapshotIdentifier']
                        result = [*result, p_dict]
        except Exception as e:
            pass
    return CheckOutput(status=CheckOutputStatus.SUCCESS,
                       objects=result,
                       error=str(""))


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(outputName="publicly_accessible_snapshots")
task.configure(inputParamsJson='''{
    "region": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "region",
    "iter_parameter": "region"
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_publicly_accessible_db_snapshots, lego_printer=aws_get_publicly_accessible_db_snapshots_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-Public-DB-Snapshots">Create List of Public DB Snapshots</h3>
<p>This action filters regions that have no manual DB snapshots and creates a list those that have</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output:&nbsp;<code>all_public_snapshots</code></p>
</blockquote>

In [9]:
all_public_snapshots = []
dummy = []
for k,v in publicly_accessible_snapshots.items():
    if v.status==CheckOutputStatus.SUCCESS:
        if len(v.objects)!=0:
            dummy = v.objects
            for x in dummy:
                all_public_snapshots.append(x)
print(all_public_snapshots)
task.configure(outputName="all_public_snapshots")

<h3 id="Change-the-public-access-to-private">Change the public access to private</h3>
<p>Using unSkript's Modify Publicly Accessible RDS Snapshots action we will modify the access to all the publicly accessible snapshots from the <em>public</em> to <em>private</em>.</p>
<blockquote>
<p>This action takes the following parameters: <code>region</code>, <code>db_snapshot_identifier</code></p>
</blockquote>

In [10]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
import pprint

from beartype import beartype
@beartype
def aws_modify_public_db_snapshots_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_modify_public_db_snapshots(handle, db_snapshot_identifier: str, region: str) -> List:
    """aws_modify_public_db_snapshots lists of publicly accessible DB Snapshot Idntifier Info.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type db_snapshot_identifier: string
        :param db_snapshot_identifier: DB Snapshot Idntifier of RDS.

        :type region: string
        :param region: Region of the RDS.

        :rtype: List with Dict of DB Snapshot Idntifier Info.
    """


    ec2Client = handle.client('rds', region_name=region)
    result = []
    try:
        response = ec2Client.modify_db_snapshot_attribute(
            DBSnapshotIdentifier=db_snapshot_identifier,
            AttributeName='restore',
            ValuesToRemove=['all'])

        result.append(response)

    except Exception as error:
        result.append(error)

    return result


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "iter.get(\\"region\\")",
    "db_snapshot_identifier": "iter.get(\\"open_snapshot\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_public_snapshots",
    "iter_parameter": ["region","db_snapshot_identifier"]
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_modify_public_db_snapshots, lego_printer=aws_modify_public_db_snapshots_printer, hdl=hdl, args=args)

<h3 id="Conclusion">Conclusion</h3>
<p>In this Runbook, we were able to secure all the publicly accessible AWS RDS DB Snapshots by using unSkript's AWS actions. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>